In [1]:
import os
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

In [36]:
iou_thresh_hold = 0.5 #FIX ME

In [2]:
def extract_detection_info(paths):
    list_info=[]
    for path in paths:
        img_name = path.split('/')[-1].split('.')[0]
    
        with open(path) as f:
            img_boxes = f.readlines()
    
        for box in img_boxes:
            box_split = box.split(' ')
        
            category = box_split[0]
            score = box_split[1]
            xmin = box_split[2]
            ymin = box_split[3]
            xmax = box_split[4]
            ymax = box_split[5].replace('\n','')
            list_info.append([img_name, category, score, xmin, ymin, xmax, ymax])
    
    return list_info

def extract_groundtruth_info(paths):
    list_info=[]
    for path in paths:
        img_name = path.split('/')[-1].split('.')[0]
    
        with open(path) as f:
            img_boxes = f.readlines()
    
        for box in img_boxes:
            box_split = box.split(' ')
        
            category = box_split[0]
            xmin = box_split[1]
            ymin = box_split[2]
            xmax = box_split[3]
            ymax = box_split[4].replace('\n','')
            list_info.append([img_name, category, xmin, ymin, xmax, ymax])
    
    return list_info

In [3]:
def classified_by_class(list_boxex_info, class_names):
    dict_classified_by_class = {}
    
    for class_name in class_names:
        dict_classified_by_class[class_name]=[]
        for box_info in list_boxex_info:
            if box_info[1]==class_name:
                dict_classified_by_class[class_name].append(box_info)
    
    return dict_classified_by_class

In [28]:
def data_distribution(dict_classified_by_class, class_names):
    x=[]
    y=[]
    for class_name in class_names:
        x.append(class_name)
        y.append(len(dict_classified_by_class[class_name]))
        print("{}: {}".format(class_name, len(dict_classified_by_class[class_name])))
    
    
    fig = plt.figure(figsize=(10, 10))
    pos = np.arange(len(class_names))
    rects = plt.bar(pos, y, align='center')
    plt.xticks(pos, x)
    plt.title("data distribution")
    plt.xlabel("class")
    plt.ylabel("count")
    plt.legend()
    fig.savefig("./results_jp/{}.png".format('data_distribution'))
#     plt.show()
    
    return print("plot save")
    

In [5]:
def sort_by_score_detection(dict_classified_by_class, class_names):
    sort_classified_by_class = {}
    for class_name in class_names:
        classify_by_class = dict_classified_by_class[class_name]
        extract_score = []
        sort_list = []
        
        for i in range(len(classify_by_class)):
            extract_score.append(classify_by_class[i][2])
        
        arr_extract_score = np.asarray(extract_score)
        arr_sort_score = np.argsort(arr_extract_score)
        
        for i in range(len(arr_sort_score)):
            sort_list.append(classify_by_class[arr_sort_score[i]])
        
        sort_classified_by_class[class_name] =  sort_list
        #dict_classified_by_class[]
    return sort_classified_by_class
    
    

In [6]:
def iou(b1, b2):

    intersection_w = max(0, min(b1[2], b2[2]) - max(b1[0], b2[0]) + 1)
    if intersection_w == 0:
        return 0
    intersection_h = max(0, min(b1[3], b2[3]) - max(b1[1], b2[1]) + 1)
    if intersection_h == 0:
        return 0
    intersection = intersection_w * intersection_h

    union = (b1[2]-b1[0]+1) * (b1[3]-b1[1]+1) + (b2[2]-b2[0]+1) * (b2[3]-b2[1]+1) - intersection

    if union == 0:
        raise ValueError('Union value must not be a zero or negative number. (boxes: {}, {})'.format(b1, b2))

    return intersection / union

In [31]:
def cal_ap(detection, groundtruth, class_names):
    dict_ap = {}
    all_true_predict=0
    all_predict=0
    all_actual=0
#     class_names=['car']
    for class_name in class_names:
        dict_ap[class_name]=[]
        
        true_predict = 0
        predict = 0
        actual = 0
        
        class_detection = detection[class_name]
        class_groundtruth = groundtruth[class_name]
        
        actual = len(class_groundtruth)
        all_actual += len(class_groundtruth)
        for i in range(len(class_detection)):
            recall=0        
            precision=1
            if predict==0:
                precision=0
            else:
                precision=true_predict/predict
            recall=true_predict/actual
        
            dict_ap[class_name].append([recall, precision])
            
            predict += 1
            all_predict += 1
            
            last_index = len(class_detection)-1
            
            img_name=class_detection[last_index-i][0]
            
            xmin=float(class_detection[last_index-i][3])
            ymin=float(class_detection[last_index-i][4])
            xmax=float(class_detection[last_index-i][5])
            ymax=float(class_detection[last_index-i][6])
            
            detection_bbox = [xmin, ymin, xmax, ymax]
            
            for j in range(len(class_groundtruth)):
                if class_groundtruth[j][0]==img_name:
                
                    x_min=float(class_groundtruth[j][2])
                    y_min=float(class_groundtruth[j][3])
                    x_max=float(class_groundtruth[j][4])
                    y_max=float(class_groundtruth[j][5])
            
                    groundtruth_bbox = [x_min, y_min, x_max, y_max]     
                
                    if iou(detection_bbox, groundtruth_bbox) >= iou_thresh_hold:
                        true_predict += 1
                        all_true_predict += 1
                        break         
        print(class_name, true_predict, predict, actual)
    print(all_true_predict, all_predict, all_actual)
    return dict_ap

In [8]:
def make_plot(result_cal_ap, class_name):
    x=[0]
    y=[1]
    
    for i in range(len(result_cal_ap)-1):
        x.append(result_cal_ap[i+1][0])
        y.append(result_cal_ap[i+1][1])
    
    fig = plt.figure(figsize=(10, 10))
    plt.plot(x, y, label=class_name)
    plt.title("AP : {}".format(class_name))
    plt.xlabel("recall")
    plt.ylabel("precision")
    plt.legend()
    fig.savefig("./results_jp/{}.png".format(class_name))
#     plt.show()
    
    return print("plot save")
    
    

In [9]:
def cal_area_practice1(result_cal_ap, class_name):
    x=[0]
    y=[1]
    
    for i in range(len(result_cal_ap)-1):
        x.append(result_cal_ap[i+1][0])
        y.append(result_cal_ap[i+1][1])

    sum_area=0
    max_y=0
    index=0
    index2=0
    for i in range(len(x)):
        if y[i]>=max_y:
            max_y=y[i]
            index=i
        else:
            
            sum_area+=max_y*(x[index]-x[index2])
            index2=index
            max_y=y[i]
            index = i
    print("{}: {}".format(class_name, sum_area))

In [10]:
def cal_area(result_cal_ap, class_name):
    x=[0]
    y=[1]
    
    for i in range(len(result_cal_ap)-1):
        x.append(result_cal_ap[i+1][0])
        y.append(result_cal_ap[i+1][1])
        
    sum_area=0
    max_y=0
    index=0
    index2=0
    for i in range(len(x)-1):
        w = x[i+1]-x[i]
        h = y[i+1]
        
        sum_area+=w*h
    print("{}: {}".format(class_name, sum_area))
    return sum_area

In [12]:
detection_save_path = os.path.join(os.getcwd(), 'detections')
groundtruth_save_path = os.path.join(os.getcwd(), 'groundtruths')

In [13]:
detection_info_paths = glob.glob(detection_save_path+'/*.txt')
groundtruth_info_paths = glob.glob(groundtruth_save_path+'/*.txt')

In [14]:
print(detection_info_paths[:3])
print(groundtruth_info_paths[:3])

['/home/intern/workspace/jp_workspace/Object-Detection-Metrics/detections/b62279e2-257b5cc0.txt', '/home/intern/workspace/jp_workspace/Object-Detection-Metrics/detections/be0a188d-c47bac51.txt', '/home/intern/workspace/jp_workspace/Object-Detection-Metrics/detections/b7a8e795-779e8c02.txt']
['/home/intern/workspace/jp_workspace/Object-Detection-Metrics/groundtruths/b62279e2-257b5cc0.txt', '/home/intern/workspace/jp_workspace/Object-Detection-Metrics/groundtruths/be0a188d-c47bac51.txt', '/home/intern/workspace/jp_workspace/Object-Detection-Metrics/groundtruths/b7a8e795-779e8c02.txt']


In [15]:
classes_path = '/home/intern/workspace/jp_workspace/keras-YOLOv3-mobilenet/model_data/uber_classes2.txt'

In [16]:
list_detection_info = extract_detection_info(detection_info_paths)
list_groundtruth_info = extract_groundtruth_info(groundtruth_info_paths)

In [17]:
print(list_detection_info[:2])
print(list_groundtruth_info[:2])

[['b62279e2-257b5cc0', 'car', '0.78574175', '396.0', '316.0', '559.0', '448.0'], ['b62279e2-257b5cc0', 'car', '0.79390377', '612.0', '329.0', '631.0', '352.0']]
[['b62279e2-257b5cc0', 'car', '523.233938', '329.617779', '589.471049', '397.618618'], ['b62279e2-257b5cc0', 'car', '624.549268', '329.617779', '634.34766', '336.47665']]


In [18]:
with open(classes_path) as f:
    class_names = f.readlines()
class_names = [name.strip() for name in class_names]

In [19]:
dict_classified_by_class_detection = classified_by_class(list_detection_info, class_names)
dict_classified_by_class_groundtruth = classified_by_class(list_groundtruth_info, class_names)

In [30]:
data_distribution(dict_classified_by_class_detection, class_names)

truck: 1315
bike: 84
bus: 388
car: 47968
motor: 59
person: 4327
rider: 69
trafficlight: 7935
trafficsign: 11231
train: 0
plot save


In [33]:
print(class_names)

['truck', 'bike', 'bus', 'car', 'motor', 'person', 'rider', 'trafficlight', 'trafficsign', 'train']


In [35]:
x=['truck', 'bike', 'bus', 'car', 'motor', 'person', 'rider', 'trafficlight', 'trafficsign', 'train']
y=[886,58,290,26328,30,2010,41,3092,4630,0]



fig = plt.figure(figsize=(10, 10))
pos = np.arange(len(class_names))
rects = plt.bar(pos, y, align='center')
plt.xticks(pos, x)
plt.title("data distribution")
plt.xlabel("class")
plt.ylabel("count")
plt.legend()
fig.savefig("./results_jp/{}.png".format('data_distribution2'))

In [21]:
sort_dict_classified_by_class_detection = sort_by_score_detection(dict_classified_by_class_detection,class_names)

In [22]:
sum=0
for c in class_names:
    sum+=len(sort_dict_classified_by_class_detection[c])
print(sum)

73376


In [32]:
dict_cal_ap = cal_ap(sort_dict_classified_by_class_detection, dict_classified_by_class_groundtruth, class_names)

truck 886 1315 3125
bike 58 84 751
bus 290 388 1156
car 26328 47968 57833
motor 30 59 329
person 2010 4327 9366
rider 41 69 478
trafficlight 3092 7935 12867
trafficsign 4630 11231 19475
train 0 0 8
37365 73376 105388


In [225]:
for class_name in class_names:
    make_plot(dict_cal_ap13[class_name], class_name)

plot save
plot save
plot save
plot save
plot save
plot save
plot save
plot save
plot save
plot save


In [27]:
mAP=0
for class_name in class_names:
    mAP+=cal_area(dict_cal_ap[class_name], class_name)
    
print("mAP: ", mAP/len(class_names))

truck: 0.24835139912572335
bike: 0.06489059773063029
bus: 0.22773677903992584
car: 0.3497494310223125
motor: 0.054078924768402994
person: 0.14247926334974628
rider: 0.06927576755757468
trafficlight: 0.13419619544889505
trafficsign: 0.15671452157682939
train: 0
mAP:  0.14474728796200403
